In [1]:
import ROOT as r
from bff_processor.Systogram import Systogram
import pickle
import re
import uncertainties

Welcome to JupyROOT 6.22/06


In [2]:
era = '2017'

In [3]:
outname="fits/{}_functional_plotter.pkl".format(era)
with open(outname,'rb') as f:
    processed_stuff= pickle.load(f)

In [4]:
def val2root(edges, values, std, name):
    nBins = len(edges) - 1
    th1f =  r.TH1F(name, name, nBins, edges[0], edges[-1])
    for i in range(th1f.GetNbinsX()):
        th1f.SetBinContent(i, values[i])
        th1f.SetBinError(i, std[i]**.5)
    return th1f

In [5]:
def hist2DH(hist, name):
    values = hist.values()
    variances = hist.variances()
    edges = hist.axes[0].edges
    hist = val2root(edges,values,variances**.5, name)
    return r.RooDataHist(name, name, r.RooArgList(x), hist)

In [6]:
x = r.RooRealVar('x', 'x', 110, 800)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [7]:
hist = processed_stuff['bff_list'][0]['SR1']
dh_signal_hists = []
for reg in processed_stuff['bff_list']:
    print(reg)
    for sr in ['SR1', 'SR2']:
        name = re.findall(r'(BFF.+).csv',reg['data'][0])[0]
        name = sr+'_'+name
        dh_signal_hists.append(hist2DH(reg[sr], name))
        edges = reg[sr].axes[0].edges
        for hist_name, up, down in reg[sr].systematics:
            dh_signal_hists.append(hist2DH(up, name+'_'+hist_name+'Up'))
            dh_signal_hists.append(hist2DH(down, name+'_'+hist_name+'Down'))

{'data': ['tw_2017_BFFZprimeToMuMu_M_400_dbs0p04.csv'], 'SR1': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=67.637, variance=0.107202) (WeightedSum(value=67.6389, variance=0.107205) with flow), 'SR2': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=19.9473, variance=0.0323959), 'CR10': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=56.2421, variance=0.101519) (WeightedSum(value=56.2463, variance=0.101528) with flow), 'CR20': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=8.06682, variance=0.015302), 'CR13': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR23': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR14': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR24': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: Weig

{'data': ['tw_2017_BFFZprimeToMuMu_M_200_dbs0p5.csv'], 'SR1': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=1520.32, variance=79.9098) (WeightedSum(value=1520.35, variance=79.911) with flow), 'SR2': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=431.474, variance=22.406) (WeightedSum(value=431.523, variance=22.4083) with flow), 'CR10': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=1549.31, variance=100.92) (WeightedSum(value=1549.42, variance=100.926) with flow), 'CR20': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=235.497, variance=15.9536), 'CR13': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR23': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR14': Systogram(Regular(139, 110, 800), storage=Weight()) # Sum: WeightedSum(value=0, variance=0), 'CR24': Systogram(Regula

In [8]:
import glob
files = [f for f in glob.glob('fits/*') if (era in f) and ('pkl' in f) and ('data' in f)]

In [10]:
abcd_pred = []
for file in files:
    with open(file, 'rb') as f:
        abcd_dict = pickle.load(f)
    nJets = re.findall(r'/(\d)_',file)[0]
    print(nJets)
    val_std = [(x.nominal_value, x.std_dev) for x in abcd_dict]
    value, stdev = list(zip(*val_std))
    hist = val2root(edges, value, stdev, "background_{}".format(nJets))
    abcd_pred.append(r.RooDataHist("SR{}_background".format(nJets), "Sbackground_{}".format(nJets), r.RooArgList(x),hist))

1
2


Warning in <TROOT::Append>: Replacing existing TH1: background_1 (Potential memory leak).


In [11]:
w = r.RooWorkspace("zp".format(era), "zp".format(era))

# Import abcd models
for dh in dh_signal_hists:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
for dh in abcd_pred:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
w.Print()

# S a v e   w o r k s p a c e   i n   f i l e
# -------------------------------------------

# Save the workspace into a ROOT file
w.writeToFile("era_{}_zp_ws.root".format(era))



Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04 Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuUp Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuUp
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuDown Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuDown
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagUp Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagUp
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagDown Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagDown
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDUp Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDUp
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDDown Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDDown
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Up Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Up
Name: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Down Title: SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Down
Name: SR1_B

Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_MuonSFUp Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_MuonSFUp
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_MuonSFDown Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_MuonSFDown
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_ElectronSFUp Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_ElectronSFUp
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_ElectronSFDown Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_Weight_ElectronSFDown
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jerUp Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jerUp
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jerDown Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jerDown
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jesTotalUp Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jesTotalUp
Name: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jesTotalDown Title: SR1_BFFZprimeToMuMu_M_450_dbs0p04_jesTotalDown
Name: SR2_BFFZprimeToMuMu_M_450_dbs0p04 Title: SR2_BFFZprimeToMuMu_M_450_dbs0p04
Name: SR2_BFFZprimeToMuMu_M_450_dbs0p04_Wei

Name: SR1_BFFZprimeToMuMu_M_500_dbs0p04_jesTotalDown Title: SR1_BFFZprimeToMuMu_M_500_dbs0p04_jesTotalDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04 Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Up Title: SR2_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Up
Name: SR2_BFFZprimeToMuMu_M_500

Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PuUp Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PuUp
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PuDown Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PuDown
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_BTagUp Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_BTagUp
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_BTagDown Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_BTagDown
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PUIDUp Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PUIDUp
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PUIDDown Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PUIDDown
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PDF_ISRFSR_Up Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PDF_ISRFSR_Up
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PDF_ISRFSR_Down Title: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_PDF_ISRFSR_Down
Name: SR1_BFFZprimeToMuMu_M_175_dbs0p04_Weight_MuonSFUp Title: SR1_BFFZprimeToMuMu_M_175_dbs

Name: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jerUp Title: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jerUp
Name: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jerDown Title: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jerDown
Name: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jesTotalUp Title: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jesTotalUp
Name: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jesTotalDown Title: SR1_BFFZprimeToMuMu_M_500_dbs0p5_jesTotalDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5 Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_PuUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_PuUp
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_PuDown Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_PuDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_BTagUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_BTagUp
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_BTagDown Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_BTagDown
Name: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_PUIDUp Title: SR2_BFFZprimeToMuMu_M_500_dbs0p5_Weight_P

Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_BTagUp Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_BTagUp
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_BTagDown Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_BTagDown
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PUIDUp Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PUIDUp
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PUIDDown Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PUIDDown
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PDF_ISRFSR_Up Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PDF_ISRFSR_Up
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PDF_ISRFSR_Down Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_PDF_ISRFSR_Down
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_MuonSFUp Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_MuonSFUp
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_MuonSFDown Title: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_MuonSFDown
Name: SR1_BFFZprimeToMuMu_M_150_dbs0p04_Weight_ElectronSFUp Title: SR1_BFFZp

Name: SR1_BFFZprimeToMuMu_M_350_dbs1p0_Weight_ElectronSFDown Title: SR1_BFFZprimeToMuMu_M_350_dbs1p0_Weight_ElectronSFDown
Name: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jerUp Title: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jerUp
Name: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jerDown Title: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jerDown
Name: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jesTotalUp Title: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jesTotalUp
Name: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jesTotalDown Title: SR1_BFFZprimeToMuMu_M_350_dbs1p0_jesTotalDown
Name: SR2_BFFZprimeToMuMu_M_350_dbs1p0 Title: SR2_BFFZprimeToMuMu_M_350_dbs1p0
Name: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_PuUp Title: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_PuUp
Name: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_PuDown Title: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_PuDown
Name: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_BTagUp Title: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_BTagUp
Name: SR2_BFFZprimeToMuMu_M_350_dbs1p0_Weight_BTagDown Title: SR2_BFFZprimeToMuMu_M_350_d

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_400_dbs0p

In [12]:
w.Print()


RooWorkspace(zp) zp contents

variables
---------
(x)

datasets
--------
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PuDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_BTagDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PUIDDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Up(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_PDF_ISRFSR_Down(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_MuonSFUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_MuonSFDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_ElectronSFUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_Weight_ElectronSFDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_400_dbs0p04_jerUp(x)
RooDataHist::S